# Conservation Analysis and Epitope Prediction


#### Author: C. Mazzaferro, K. Fisch
#### Email: cmazzafe@ucsd.edu
#### Date: October 2016
 
## Outline of Notebook
<a id = "toc"></a>
1. <a href = "#background">Background</a>
2. <a href = "#Cons">High Affinity Binding Prediction </a>
    * <a href = "#Agg">Data Aggregation</a>
    * <a href = "#Sim">Similarty Score Calculation</a>
    * <a href = "#Plot">Visualize</a>

### Files of interest
From a simple fasta file containing the proteins of interest, this workflow will create:

1. Binding affinity prediction xls files for the given allele, nmer combinations. Default set to all allele supertypes and [8, 9, 10, 11] nmers
2. Csv file containing all high binding affinity peptides. Threshold can be set to a customized level (either by setting it to Medium or setting it by affinity score. See [CBS's prediction services](http://www.cbs.dtu.dk/services/), in particular [netMHC](http://www.cbs.dtu.dk/services/NetMHC/) for more specific information.
3. Fasta files containing swap data: for each high affinity binding peptide, find every peptide that results from swapping one single AA from it. These fasta files are then fed into netMHC again and their binding affinity determined again. 
4. Csv files for each fasta file with the swapped peptide binding prediction.

Caveats: netMHC local install must be in the system's path. 

In [2]:
import importlib
from nepitope import net_MHC_interface
from nepitope import mhc_utils
from nepitope import pep_utils
import pandas

project_location = '/Users/carlomazzaferro/Desktop/New_General/MICE/'
fasta_file = '/Users/carlomazzaferro/Desktop/New_General/MICE/fasta_base_new_prots.fasta'
net_mhc_path = '/Users/carlomazzaferro/Desktop/BINF_Tools/netMHC-4.0/netMHC'


alleles = ['H-2-Ld', 'H-2-Db', 'H-2-Dd', 'H-2-Kb', 'H-2-Kd', 'H-2-Kk']
nmers =[8, 9, 10, 11]

net_mhc = net_MHC_interface.netMHCComand(net_mhc_path, fasta_file, nmers=nmers, alleles=alleles)

/Users/carlomazzaferro/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


### <a id = "Cons"></a>
# MHC Binding Affinity Prediction 
### Ran on netMHCcons for n-mers of 8, 9, 10, 11 amino acids and for the 12 allele supertype families

Run for each protein sequence retrieved from reference paper.

In [20]:
net_mhc.create_text_command()

'netMHC commands written to run_netMHC.txt located at /Users/carlomazzaferro/Desktop/New_General/MICE/mhc_preds_fasta_base_new_prots'

In [21]:
net_mhc.run_netMHC()

Predictions being saved to /Users/carlomazzaferro/Desktop/New_General/MICE/mhc_preds_fasta_base_new_prots


In [100]:
### Load predictions 
import importlib
importlib.reload(mhc_utils)
pred_loc = '/Users/carlomazzaferro/Desktop/New_General/MICE/mhc_preds_fasta_2_prots/'
agg = mhc_utils.FileConsolidation(pred_loc, fasta_file)

In [4]:
csv_dir = '/Users/carlomazzaferro/Desktop/New_General/MICE/high_affinity_peps_ALL.csv'
high_aa_df = agg.get_all_high_affinity_from_batch(csv_out=True, csv_dir=csv_dir)

/Users/carlomazzaferro/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [101]:
list_df = agg.return_df_list()

In [102]:
lsss = agg.optimized_list_df_by_prot(list_df)

In [104]:
pandas.concat(lsss).to_csv('/Users/carlomazzaferro/Desktop/New_General/MICE/two_prots_all_preds.csv')

In [ ]:
from nepitope import mhc_utils, summary_data
import glob
nmers = [8, 9,10,11]

#files saved from netMHCcons as split_*i*_prediction_*n*_mer for each split/nmer
filepath = '/Users/carlomazzaferro/Desktop/new/swaps/mhc_preds_per_swap/'
fasta_file = '/Users/carlomazzaferro/Desktop/Test_IEDB/newer_pipeline_files/fasta_from_study_dealigned.fasta'

file_pattern = '_swap_*'


files = glob.glob(filepath + file_pattern)
original_peps_and_pos = []

for file in files:
    split_ = file.split('/')[-1]
    split_2 = split_.split('_')
    original_peps_and_pos.append([split_2[2], split_2[4]])

In [14]:
from nepitope import summary_data
importlib.reload(summary_data)
smm = summary_data.SummaryData()
ssmm = smm.summarize_all_data(lsss, fasta_file)

In [25]:
df = pandas.read_csv('/Users/carlomazzaferro/Desktop/New_General/MICE/high_affinity_peps.csv').sort_values(by='Pos')

In [26]:
df_sp = df[df['ID'] == 'S__pyogenes_Cas9']

In [30]:
df_sp = df_sp.drop('Unnamed: 0', 1)
df_sp.to_csv('/Users/carlomazzaferro/Desktop/New_General/MICE/SP_high_aff_peps.csv')

In [58]:
dat = list(zip(list(df_sp.Pos), list(df_sp.Peptide)))

In [69]:
[153]

'I'

In [16]:
pandas.DataFrame(ssmm.data_list, index=ssmm.indexes).T

,Protein,Num High Affinity Peps,Num Med Affinity Peps,Num Low Affinity Peps,Num No Affinity Peps,Protein Length
0,S__pyogenes_Cas9,40,174,673,31741,1368
1,Staphylococcus_aureus_Cas9,32,123,553,24360,1053
2,S_CRISPR_1_thermophilus_Cas9,25,100,539,26036,1121
3,N__meningitidis_Cas9,20,89,424,25231,1082
4,Parvibaculum_lavamentivorans_Cas9,14,58,321,24291,1037
5,Corynebacter_diphtheria_Cas9,14,78,407,25313,1084
6,Streptococcus_pasteurianus_Cas9,34,101,469,26312,1130
7,Neisseria_cinerea_Cas9,22,86,442,25214,1082
8,Campylobacter_lari_Cas9,28,122,566,23152,1003
9,T__denticola_Cas9,40,173,700,32363,1395


In [1]:
high_aa_df.head()

NameError: name 'high_aa_df' is not defined

# Swap Files

In [72]:
csv_dir = '/Users/carlomazzaferro/Desktop/New_General/MICE/high_affinity_peps.csv'

In [88]:
high_aa_df = pandas.read_csv(csv_dir) 
high_aa_df = high_aa_df[high_aa_df['ID'] == 'Staphylococcus_aureus_Cas9']

In [93]:
from nepitope import peptide_utilities
importlib.reload(peptide_utilities)
fasta_files_dir = '/Users/carlomazzaferro/Desktop/swaps_2/'

swap = peptide_utilities.Swaps(high_aa_df, fasta_files_dir, net_mhc_path, proteins=['Staphylococcus_aureus_Cas9'] )

In [94]:
cmd = swap.find_swaps_write_to_fasta()

Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_preds__swap_ALIIANADFI_Pos_704_ID_Staphylococcus-aureus-Cas9_Allele_H-2-Db_nmer_10
Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_preds__swap_ITYREYLENM_Pos_998_ID_Staphylococcus-aureus-Cas9_Allele_H-2-Kb_nmer_10
Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_preds__swap_SEEEFSAALL_Pos_100_ID_Staphylococcus-aureus-Cas9_Allele_H-2-Kk_nmer_10
Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_preds__swap_EEFSAALLHL_Pos_102_ID_Staphylococcus-aureus-Cas9_Allele_H-2-Kk_nmer_10
Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_preds__swap_LEYYEKFQII_Pos_274_ID_Staphylococcus-aureus-Cas9_Allele_H-2-Kk_nmer_10
Predictions being saved to /Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/mhc_pre

In [ ]:
from nepitope import mhc_utils, summary_data
import glob
nmers = [8, 9,10,11]

#files saved from netMHCcons as split_*i*_prediction_*n*_mer for each split/nmer
filepath = '/Users/carlomazzaferro/Desktop/New_General/MICE/mhc_preds_fasta_2_prots/'
fasta_file = '/Users/carlomazzaferro/Desktop/New_General/MICE/fasta_2_prots.fasta'

file_pattern = '_swap_*'

aggregate_all = mhc_utils.FileConsolidation(filepath, fasta_file)

files = glob.glob(filepath + file_pattern)
original_peps_and_pos = []

for file in files:
    split_ = file.split('/')[-1]
    split_2 = split_.split('_')
    original_peps_and_pos.append([split_2[2], split_2[4]])

## Load file

In [95]:
from nepitope import mhc_utils
import pandas
import glob
import importlib
importlib.reload(mhc_utils)

#files saved from netMHC in  multiple different files, one per each query (nmer, allele)
filepath = '/Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/preds_per_swap/'
fasta_file = '/Users/carlomazzaferro/Desktop/New_General/MICE/fasta_base_new_prots.fasta'
aggregate_all = mhc_utils.FileConsolidation(filepath, fasta_file)

In [96]:
files = glob.glob(filepath + '*')
original_peps_and_pos = []

for file in files:
    split_ = file.split('/')[-1]
    split_2 = split_.split('_')
    original_peps_and_pos.append([split_2[2], split_2[4]])

In [97]:
#returns a dataframe per file used (just one in this case)
df_list = aggregate_all.return_df_list()

In [98]:
from nepitope import mhc_utils, summary_data

summ_data = summary_data.SummaryData()
summ_data_analysis = summ_data.summarize_data_for_each_mhc_pred(df_list, original_peps_and_pos, high_aa_df)


In [99]:
csv_path = '/Users/carlomazzaferro/Desktop/swaps_2/Staphylococcus_aureus_Cas9/summary_results_per_prediction.csv'
summ_data_analysis.summary_df.sort_values(by='original pos').to_csv(path_or_buf=csv_path, sep=',')

In [27]:
ss = summ_data_analysis.summary_df.sort_values(by='original pos')

### DFLIST FROM ALL PREDS 

In [5]:
df_list = agg.return_df_list()

In [6]:
lsss_1 = agg.optimized_list_df_by_prot(df_list)

In [7]:
from nepitope import pairwise_comp
importlib.reload(pairwise_comp)

fasta_file = '/Users/carlomazzaferro/Desktop/New_General/MICE/fasta_base_new_prots.fasta'


pwcomp = pairwise_comp.PairwiseComp(lsss_1, 5, fasta_file) 
df_pairwise_comps  = pwcomp.pipe_run() 

/Users/carlomazzaferro/Documents/Code/neoantigen/antigen_discovery/nepitope/pairwise_comp.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Peptide'] = df['Peptide'].str.replace('X', '-')


In [15]:
sa_df = pwcomp.filt_dfs[1]

In [17]:
sa_peps = sa_df.Peptide.values.tolist()

In [22]:
def _get_possible_peptides(stri):
    # TODO: Clean this up asap. Too hacky, not very readable

    nmer_list = list(range(5, len(stri) + 1))
    length_pep = len(stri)
    num_possible_sub_peps = [length_pep - nmer + 1 for nmer in nmer_list if length_pep - nmer + 1 > 0]

    subs_all = []
    for i in num_possible_sub_peps:
        for nmer, j in zip(nmer_list, range(i)):
            subs = [stri[j:j + nmer] for i, nmer in enumerate(nmer_list)]
            subs_all.extend(subs)

    subs_all = list(set(subs_all))
    lst = [[w for w in subs_all if len(w) == num] for num in set(len(i) for i in subs_all)]

    return lst

In [23]:
all_5_nmers = {}
for pep in sa_peps:
    all_5_nmers[pep] = _get_possible_peptides(pep)

In [48]:
finder = []
for i in list(all_5_nmers.values()):
    for j in i:
        for k in j:
            if prot_sp.find(k) != -1:
                finder.append(prot_sp.find(k))

In [51]:
finder

[346, 346, 346]

In [50]:
sp_df.Range

152       [[152, 153, 154, 155, 156, 157, 158, 159, 160]]
153       [[153, 154, 155, 156, 157, 158, 159, 160, 161]]
367       [[367, 368, 369, 370, 371, 372, 373, 374, 375]]
368            [[368, 369, 370, 371, 372, 373, 374, 375]]
368     [[368, 369, 370, 371, 372, 373, 374, 375, 376,...
369     [[369, 370, 371, 372, 373, 374, 375, 376, 377,...
369     [[369, 370, 371, 372, 373, 374, 375, 376, 377,...
385            [[385, 386, 387, 388, 389, 390, 391, 392]]
439       [[439, 440, 441, 442, 443, 444, 445, 446, 447]]
515       [[515, 516, 517, 518, 519, 520, 521, 522, 523]]
520     [[520, 521, 522, 523, 524, 525, 526, 527, 528,...
521       [[521, 522, 523, 524, 525, 526, 527, 528, 529]]
564            [[564, 565, 566, 567, 568, 569, 570, 571]]
687       [[687, 688, 689, 690, 691, 692, 693, 694, 695]]
688            [[688, 689, 690, 691, 692, 693, 694, 695]]
807       [[807, 808, 809, 810, 811, 812, 813, 814, 815]]
891       [[891, 892, 893, 894, 895, 896, 897, 898, 899]]
906     [[906,

In [16]:
sp_df = pwcomp.filt_dfs[0]

In [36]:
prot_sp = pwcomp.peps_prots_ranges[0][1]

In [8]:
ref = 'S__pyogenes_Cas9'
df_pairwise_comps.loc[ref]

,10,11,5,6,7,8,9,Matches Loc,Num High AA,Total
C__jejuni_Cas9,0,0,0,0,0,0,0,[[]],0,0
Campylobacter_lari_Cas9,0,0,3,0,0,0,0,[[]],0,3
Corynebacter_diphtheria_Cas9,0,0,0,0,0,0,0,[[]],0,0
F__novicida_Cas9,0,0,0,0,0,0,0,[[]],0,0
Lactobacillus_buchneri_Cas9,0,0,1,0,0,0,0,[[]],0,1
Listeria_innocua_Cas9,0,0,26,14,4,1,0,"[[[155, 156, 157, 158, 159], [154, 155, 156, 1...",23,45
N__meningitidis_Cas9,0,0,0,0,0,0,0,[[]],0,0
Neisseria_cinerea_Cas9,0,0,3,0,0,0,0,[[]],0,3
P__multocida_Cas9,0,0,0,0,0,0,0,[[]],0,0
Parvibaculum_lavamentivorans_Cas9,0,0,0,0,0,0,0,[[]],0,0


In [45]:
df_pairwise_comps.to_csv('/Users/carlomazzaferro/Desktop/New_General/MICE/df_pairwise_comps.csv')

In [140]:
swaps_df_dir = '/Users/carlomazzaferro/Desktop/Test_IEDB/OtherProtsPrashant/swap_results/S__pyogenes_Cas9/summary_results_per_prediction.csv'
swaps_df = pandas.read_csv(swaps_df_dir)

In [314]:
from importlib import reload
reload(merge_workflows)
from nepitope import merge_workflows
merged = merge_workflows.MergeSwapsAndComp(df_pairwise_comps, swaps_df, 40)


/Users/carlomazzaferro/Documents/Code/neoantigen/antigen_discovery/nepitope/merge_workflows.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df = pandas.DataFrame(maximal_list)


In [577]:
merged.top_loc_matches[0:20]
for i in merged.top_loc_matches[0:20]:
    print('%i matches occurring at position %i' %(i[1], i[0]))

130 matches occurring at position 657
129 matches occurring at position 447
126 matches occurring at position 656
124 matches occurring at position 446
122 matches occurring at position 658
119 matches occurring at position 448
117 matches occurring at position 813
115 matches occurring at position 814
110 matches occurring at position 659
109 matches occurring at position 445
109 matches occurring at position 812
106 matches occurring at position 514
106 matches occurring at position 655
103 matches occurring at position 513
100 matches occurring at position 449
98 matches occurring at position 512
98 matches occurring at position 515
98 matches occurring at position 985
98 matches occurring at position 987
96 matches occurring at position 986


In [315]:
df_top = merged.top_swap_df
df_ordered_filtered = merged.final_sort_and_filtering(df_top)

/Users/carlomazzaferro/Documents/Code/neoantigen/antigen_discovery/nepitope/merge_workflows.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [642]:
df_ordered_filtered.columns

Index(['Unnamed: 0', 'nmer', 'allele', 'num high affinity peps',
       'num med affinity peps', 'num low affinity peps',
       'num no affinity peps', 'original peptide', 'original pos',
       'top scoring peptides', 'Range'],
      dtype='object')

In [610]:
reload(modify_data)
from nepitope import modify_data
mod_fasta = '/Users/carlomazzaferro/Desktop/PROJECT_LOC_TEST/mod_fastas/mod_all_fastas/'
orig_fasta = '/Users/carlomazzaferro/Desktop/PROJECT_LOC_TEST/prots_in_fasta.fasta'

mod_data = modify_data.ModifyData(swaps_df, fasta_file, mod_fasta, lsss_1)

In [611]:
######## TO INCLUDE IN TEST SUITE ########

from nepitope import pep_utils
j,k = pep_utils.create_separate_lists(fasta_file)
for i in sgl:
    print(i[1] in k[0])

for i in mod_data.swaps_dic.keys():
    print(i in mod_data.fasta_dic[0][1])
    print(i)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
RRYTRRKNR
True
KAILSARLSK
True
KLNREDLLRK
True
WRQLLNAKLI
True
SRFAWMTRK
True
LIHDDSLTF
True
IYLALAHMI
True
YHHAHDAYL
True
SLLYEYFTVY
True
ILLSDILRV
True
MLASAGELQK
True
KQSGKTILDF
True
RRLENLIAQL
True
KTYAHLFDDK
True
AHMIKFRGHF
True
LPKYSLFEL
True
SLHEHIANL
True
SPAIKKGIL
True
RLIYLALAHM
True
HYLDEIIEQI
True
RRKNRICYL
True
REQAENIIHL
True
KVLPKHSLLY
True
MIKFRGHFL
True
LLYEYFTVY
True
ALSLGLTPNF
True
NGL

In [612]:
mod_data.get_modified_fastas_single_swap()

In [615]:
exchange_pairs = []
for i,j in mod_data.swaps_dic.items():
    exchange_pairs.append([i,j])
    

In [603]:
import random

def get_all_exchange_pairs(swaps_df):
    cols = ['original peptide', 'top scoring peptides']
    sliced = swaps_df.ix[:, cols].values.tolist()
    orig = [i[0] for i in sliced]
    swaps = [list(filter(None, re.sub('[^A-Za-z0-9]+', ',', i[1]).split(','))) for i in sliced]
    hashmap = [[orig[i], swaps[i]] for i in range(len(orig))]
    swaps_single = []
    for idx, hsh in enumerate(hashmap):
        swaps_single.append([hsh[0], random.choice(hsh[1])])
    return swaps_single

sgl = get_all_exchange_pairs(swaps_df)



In [616]:
import os
fastas = glob.glob(mod_fasta + '*')
csv_path = '/Users/carlomazzaferro/Desktop/PROJECT_LOC_TEST/mod_csvs/mod_csvs_all_peps/'

list_dfs = []
for pair in exchange_pairs:
    for fasta in fastas:
        if os.path.splitext(os.path.basename(fasta))[0].split('-')[0] == pair[0]:
            my_fasta = fasta
            ls_mod = mod_data.get_singly_modified_df_list(pair[0], pair[1])
            pwcomp_pair_swap = pairwise_comp.PairwiseComp(ls_mod, 5, my_fasta)
            df_swap = pwcomp_pair_swap.pipe_run()
            df_swap = df_swap.drop('Matches Loc', 1)
            df_swap.to_csv(csv_path + pair[0] + '-' + pair[1] + '.csv')
            list_dfs.append(df_swap)
            
            
    


/Users/carlomazzaferro/Documents/Code/neoantigen/antigen_discovery/nepitope/pairwise_comp.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Peptide'] = df['Peptide'].str.replace('X', '-')


In [629]:
list_dfs[97].loc['S__pyogenes_Cas9']

,10,11,5,6,7,8,9,Num High AA,Total
C__jejuni_Cas9,0,0,0,0,0,0,0,0,0
Campylobacter_lari_Cas9,0,0,0,0,0,0,0,0,0
Corynebacter_diphtheria_Cas9,0,0,0,0,0,0,0,0,0
F__novicida_Cas9,0,0,0,0,0,0,0,0,0
Lactobacillus_buchneri_Cas9,0,0,15,5,2,1,0,2,23
Listeria_innocua_Cas9,0,0,67,41,23,13,5,7,149
N__meningitidis_Cas9,0,0,2,0,0,0,0,0,2
Neisseria_cinerea_Cas9,0,0,2,0,0,0,0,0,2
P__multocida_Cas9,0,0,2,1,0,0,0,0,3
Parvibaculum_lavamentivorans_Cas9,0,0,0,0,0,0,0,0,0


In [630]:
list_dfs[74].loc['S__pyogenes_Cas9']

,10,11,5,6,7,8,9,Num High AA,Total
C__jejuni_Cas9,0,0,0,0,0,0,0,0,0
Campylobacter_lari_Cas9,0,0,0,0,0,0,0,0,0
Corynebacter_diphtheria_Cas9,0,0,0,0,0,0,0,0,0
F__novicida_Cas9,0,0,0,0,0,0,0,0,0
Lactobacillus_buchneri_Cas9,0,0,15,5,2,1,0,4,23
Listeria_innocua_Cas9,0,0,67,41,23,13,5,8,149
N__meningitidis_Cas9,0,0,2,0,0,0,0,0,2
Neisseria_cinerea_Cas9,0,0,2,0,0,0,0,0,2
P__multocida_Cas9,0,0,2,1,0,0,0,0,3
Parvibaculum_lavamentivorans_Cas9,0,0,0,0,0,0,0,0,0


In [165]:
pwcomp_1_swap = pairwise_comp.PairwiseComp(ls_mod, 5, mod_fasta)

/Users/carlomazzaferro/Documents/Code/neoantigen/antigen_discovery/nepitope/pairwise_comp.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Peptide'] = df['Peptide'].str.replace('X', '-')


In [166]:
df_1_swap = pwcomp_1_swap.pipe_run()

In [156]:
df_1_swap.to_csv('/Users/carlomazzaferro/Desktop/Test_IEDB/OtherProtsPrashant/single_swap_pairwise_comp.csv')

In [168]:
df_1_swap.loc['S__pyogenes_Cas9']

,10,11,5,6,7,8,9,Matches Loc,Num High AA,Total
C__jejuni_Cas9,0,0,0,0,0,0,0,[[]],0,0
Campylobacter_lari_Cas9,0,0,0,0,0,0,0,[[]],0,0
Corynebacter_diphtheria_Cas9,0,0,0,0,0,0,0,[[]],0,0
F__novicida_Cas9,0,0,0,0,0,0,0,[[]],0,0
Lactobacillus_buchneri_Cas9,0,0,13,5,2,1,0,"[[[1266, 1267, 1268, 1269, 1270], [1266, 1267,...",5,21
Listeria_innocua_Cas9,0,0,66,40,23,13,5,"[[[8, 9, 10, 11, 12, 13, 14, 15], [7, 8, 9, 10...",118,147
N__meningitidis_Cas9,0,0,2,0,0,0,0,"[[[7, 8, 9, 10, 11], [7, 8, 9, 10, 11]]]",2,2
Neisseria_cinerea_Cas9,0,0,2,0,0,0,0,"[[[7, 8, 9, 10, 11], [7, 8, 9, 10, 11]]]",2,2
P__multocida_Cas9,0,0,2,1,0,0,0,[[]],0,3
Parvibaculum_lavamentivorans_Cas9,0,0,0,0,0,0,0,[[]],0,0


In [161]:
df_pairwise_comps.loc['S__pyogenes_Cas9']

,10,11,5,6,7,8,9,Matches Loc,Num High AA,Total
C__jejuni_Cas9,0,0,0,0,0,0,0,[[]],0,0
Campylobacter_lari_Cas9,0,0,0,0,0,0,0,[[]],0,0
Corynebacter_diphtheria_Cas9,0,0,0,0,0,0,0,[[]],0,0
F__novicida_Cas9,0,0,0,0,0,0,0,[[]],0,0
Lactobacillus_buchneri_Cas9,0,0,15,5,2,1,0,"[[[1266, 1267, 1268, 1269, 1270], [1266, 1267,...",7,23
Listeria_innocua_Cas9,0,0,67,41,23,13,5,"[[[8, 9, 10, 11, 12, 13, 14, 15], [7, 8, 9, 10...",120,149
N__meningitidis_Cas9,0,0,2,0,0,0,0,"[[[7, 8, 9, 10, 11], [7, 8, 9, 10, 11]]]",2,2
Neisseria_cinerea_Cas9,0,0,2,0,0,0,0,"[[[7, 8, 9, 10, 11], [7, 8, 9, 10, 11]]]",2,2
P__multocida_Cas9,0,0,2,1,0,0,0,[[]],0,3
Parvibaculum_lavamentivorans_Cas9,0,0,0,0,0,0,0,[[]],0,0
